In [1]:
import datatable as dt
import pandas as pd
import xgboost as xgb

- copy from [Yirun Zhang](https://www.kaggle.com/gogo827jz/jane-street-xgboost-grouptimesplitkfold)

In [2]:
%%time
train_data_datatable = dt.fread('~/env/kaggle/janestreet/train.csv')
train_data = train_data_datatable.to_pandas()

CPU times: user 29.6 s, sys: 18.2 s, total: 47.8 s
Wall time: 10.2 s


In [3]:
%%time
train = train_data.query('weight > 0').reset_index(drop = True)
train['action'] = (train['resp'] > 0).astype('int')
train.describe()


CPU times: user 10.3 s, sys: 2.64 s, total: 12.9 s
Wall time: 13.3 s


,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id,action
count,1.981287e+06,1.981287e+06,1.981287e+06,1.981287e+06,1.981287e+06,1.981287e+06,1.981287e+06,1.981287e+06,1.981287e+06,1.981287e+06,...,1.981139e+06,1.981139e+06,1.969414e+06,1.969414e+06,1.974795e+06,1.974795e+06,1.979918e+06,1.979918e+06,1.981287e+06,1.981287e+06
mean,2.526263e+02,3.657652e+00,1.519922e-04,2.124634e-04,3.074072e-04,5.020192e-04,4.721902e-04,5.960267e-03,5.035693e-01,4.236199e-01,...,4.582973e-01,1.754965e-01,4.503705e-01,1.644199e-01,5.067359e-01,1.590565e-01,4.812592e-01,1.604972e-01,1.216696e+06,5.044130e-01
std,1.512144e+02,8.291001e+00,9.324476e-03,1.285641e-02,1.991720e-02,3.436747e-02,2.816607e-02,9.999825e-01,2.623796e+00,2.513707e+00,...,2.116926e+00,1.897661e+00,1.765243e+00,2.182138e+00,2.559638e+00,1.748438e+00,2.336166e+00,1.712709e+00,6.853098e+05,4.999807e-01
min,0.000000e+00,6.912442e-03,-3.675043e-01,-5.328334e-01,-5.681196e-01,-5.987447e-01,-5.493845e-01,-1.000000e+00,-3.172026e+00,-3.093182e+00,...,-5.647779e+00,-6.029281e+00,-4.080720e+00,-8.136407e+00,-7.903043e+00,-5.765982e+00,-7.024909e+00,-5.129002e+00,1.000000e+00,0.000000e+00
25%,1.130000e+02,3.786683e-01,-1.939318e-03,-2.777549e-03,-5.248402e-03,-9.680324e-03,-7.466364e-03,-1.000000e+00,-1.195116e+00,-1.206613e+00,...,-1.028427e+00,-1.012458e+00,-8.189171e-01,-1.245990e+00,-1.299109e+00,-9.358700e-01,-1.163105e+00,-9.021064e-01,6.300985e+05,0.000000e+00
50%,2.610000e+02,1.072844e+00,4.759053e-05,7.172624e-05,1.235709e-04,1.353374e-04,9.596080e-05,1.000000e+00,1.572825e-01,1.039031e-01,...,1.379693e-01,-9.741442e-02,1.448302e-01,-6.605669e-02,2.072707e-01,-6.179322e-02,1.693839e-01,-7.633027e-02,1.226025e+06,1.000000e+00
75%,3.850000e+02,3.196557e+00,2.193924e-03,3.078691e-03,5.710722e-03,1.023963e-02,7.903929e-03,1.000000e+00,1.756368e+00,1.645332e+00,...,1.555857e+00,1.181069e+00,1.366768e+00,1.301766e+00,1.958822e+00,1.018834e+00,1.751426e+00,1.019705e+00,1.810380e+06,1.000000e+00
max,4.990000e+02,1.672937e+02,2.453477e-01,2.949339e-01,3.265597e-01,5.113795e-01,4.484616e-01,1.000000e+00,7.442989e+01,1.480763e+02,...,4.812516e+01,1.241175e+02,6.514517e+01,7.052807e+01,5.872849e+01,6.932221e+01,5.119038e+01,1.164568e+02,2.390489e+06,1.000000e+00


In [4]:
features = [c for c in train.columns if 'feature' in c]